In [38]:
import sqlite3
import pandas as pd
import os
from datetime import datetime

In [39]:
# Connect to SQLite DB
with sqlite3.connect("../weather.db") as conn:
    cursor = conn.cursor()
    
    cursor.executescript("""
    -- Dimension: Time
    CREATE TABLE IF NOT EXISTS dim_tiempo (
        fecha_id INTEGER PRIMARY KEY AUTOINCREMENT,
        fecha TEXT UNIQUE,
        año INTEGER,
        mes INTEGER,
        día INTEGER,
        nombre_mes TEXT,
        trimestre INTEGER
    );

    -- Dimension: Station
    CREATE TABLE IF NOT EXISTS dim_estacion (
        estacion_id INTEGER PRIMARY KEY AUTOINCREMENT,
        nombre_estacion TEXT,
        provincia TEXT,
        latitud REAL,
        longitud REAL,
        UNIQUE (nombre_estacion, provincia)
    );

    -- Dimension: Wind Direction
    CREATE TABLE IF NOT EXISTS dim_direccion_viento (
        wdir_id INTEGER PRIMARY KEY AUTOINCREMENT,
        direccion_cardinal TEXT UNIQUE
    );

    -- Fact Table
    CREATE TABLE IF NOT EXISTS hechos_clima (
        id_hecho INTEGER PRIMARY KEY AUTOINCREMENT,
        fecha_id TEXT,
        estacion_id INTEGER,
        wdir_id INTEGER,
        tmax REAL,
        tmin REAL,
        tavg REAL,
        prcp REAL,
        wspd REAL,
        UNIQUE (fecha_id, estacion_id, wdir_id),
        FOREIGN KEY (fecha_id) REFERENCES dim_tiempo(fecha_id),
        FOREIGN KEY (estacion_id) REFERENCES dim_estacion(estacion_id),
        FOREIGN KEY (wdir_id) REFERENCES dim_direccion_viento(wdir_id)
    );

    -- Indexes for optimization
    CREATE INDEX IF NOT EXISTS idx_hechos_fecha_id ON hechos_clima(fecha_id);
    CREATE INDEX IF NOT EXISTS idx_hechos_estacion_id ON hechos_clima(estacion_id);
    CREATE INDEX IF NOT EXISTS idx_hechos_wdir_id ON hechos_clima(wdir_id);
    """)

In [40]:
def select_or_insert_dim_tiempo(cursor, row):
    fecha_str = row["Date"]
    cursor.execute("SELECT fecha_id FROM dim_tiempo WHERE fecha = ?", (fecha_str,))
    row_db = cursor.fetchone()
    if row_db:
        return row_db[0]

    fecha = datetime.strptime(fecha_str, "%Y-%m-%d")
    cursor.execute("""
        INSERT INTO dim_tiempo (fecha, año, mes, día, nombre_mes, trimestre)
        VALUES (?, ?, ?, ?, ?, ?)
    """, (
        fecha_str,
        fecha.year,
        fecha.month,
        fecha.day,
        fecha.strftime("%B"),
        (fecha.month - 1) // 3 + 1
    ))

    return cursor.lastrowid

In [41]:
def select_or_insert_dim_estacion(cursor, nombre, provincia, latitud, longitud):
    cursor.execute("SELECT estacion_id FROM dim_estacion WHERE nombre_estacion = ? AND provincia = ?", (nombre, provincia))
    row = cursor.fetchone()
    if row:
        return row[0]
    
    cursor.execute("INSERT INTO dim_estacion (nombre_estacion, provincia, latitud, longitud) VALUES (?, ?, ?, ?)", (nombre, provincia, latitud, longitud))
    return cursor.lastrowid


In [42]:
def select_or_insert_dim_direccion_viento(cursor, direccion):
    direccion = direccion.upper() if isinstance(direccion, str) else None
    if direccion is not None:
        cursor.execute("SELECT wdir_id FROM dim_direccion_viento WHERE direccion_cardinal = ?", (direccion,))
    else:
        cursor.execute("SELECT wdir_id FROM dim_direccion_viento WHERE direccion_cardinal IS NULL")
    
    row = cursor.fetchone()
    if row:
        return row[0]

    cursor.execute(
        "INSERT INTO dim_direccion_viento (direccion_cardinal) VALUES (?)",
        (direccion,)
    )
    return cursor.lastrowid


In [43]:
# Merge all data sources
csv_files = [
    "datos_limpios_bocas_del_toro_2024.csv",
    "datos_limpios_chiriqui_2024.csv",
    "datos_limpios_santiago_2024.csv",
    "datos_limpios_tocumen_2024.csv",
]

output_file = "main_dataset.csv"
output_dir = "../datos_limpios"
output = f"{output_dir}/{output_file}"
if os.path.exists(output):
    os.remove(output)

# Merge all files
first = True
for file in csv_files:
    path = os.path.join(output_dir, file)
    for chunk in pd.read_csv(path, chunksize=1000):
        chunk.to_csv(output, mode='a', index=False, header=first)
        first = False

In [44]:
# Load CSV
csv_path = '../datos_limpios/main_dataset.csv'  # Replace with the path to your CSV
df = pd.read_csv(csv_path)

In [45]:
points = {
    "Bocas del Toro": (9.3400, -82.2400, 11),
    "Veraguas": (8.1000, -80.9833, 90),
    "Panamá": (9.08939000, -79.38310000),
    "Chiriquí": (8.42729000, -83.43085001)
}

def get_coordinates(location: str):
    return points[location]

In [50]:
# Loop through merged CSV
with sqlite3.connect("../weather.db") as conn:
    cursor = conn.cursor()
    
    for chunk in pd.read_csv(output, chunksize=512):
        for _, row in chunk.iterrows():
            coordinates = get_coordinates(row["Provincia"])
            fecha_id = select_or_insert_dim_tiempo(cursor, row)
            estacion_id = select_or_insert_dim_estacion(cursor, row["Estacion"], row["Provincia"], coordinates[0], coordinates[1])
            wdir_id = select_or_insert_dim_direccion_viento(cursor, row["Wdir"])

            # Ignore errors if running this code block again
            cursor.execute("""
                INSERT OR IGNORE INTO hechos_clima (
                    fecha_id, estacion_id, wdir_id, tmax, tmin, tavg, prcp, wspd
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            """, (
                fecha_id,
                estacion_id,
                wdir_id,
                row["Tmax"],
                row["Tmin"],
                row["Tavg"],
                row["Prcp"],
                row["Wspd"]
            ))